# Langchain on Vertex AI - Mamang Chatbot Use Case

This jupyter notebook will be used to deploy a chatbot built using the Langchain model on Vertex AI. 

In [2]:
# installing dependencies

!pip install --upgrade --quiet \
    google-cloud-aiplatform \
    langchain \
    langchain-google-vertexai \
    cloudpickle \
    pydantic \
    langchain_google_community \
    google-cloud-discoveryengine \
    google-api-python-client \
    google-auth

In [3]:
# setting up few variables

PROJECT_ID = "imrenagi-gemini-experiment"
LOCATION = "us-central1"  
STAGING_BUCKET = "gs://courses-imrenagicom-agent"  
DATA_STORE_ID = "course-software-instrumentation_1719363827721"
LOCATION_ID = "global"
LLM_MODEL = "gemini-1.5-pro-001"

In [4]:
# importing necessary libraries since I'm just an importir

from IPython.display import display, Markdown

from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents import tool

from langchain.memory import ChatMessageHistory
from langchain_google_community import VertexAISearchRetriever
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

import vertexai
from vertexai.preview import reasoning_engines

from typing import List

In [5]:
# Initializing vertex AI 

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

In [6]:
## Model safety settings
safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

## Model parameters
model_kwargs = {
    "temperature": 0.5,
    "safety_settings": safety_settings,
}

## Search Course Content Tool

This tool will be used to search for course content based on the user's query and execute query to Vertex AI Search datastore. 

In [14]:
@tool
def search_course_content(query: str) -> str:
    """Explain about software instrumentation course materials."""
    from langchain_google_community import VertexAISearchRetriever

    retriever = VertexAISearchRetriever(
        project_id=PROJECT_ID,
        data_store_id=DATA_STORE_ID,
        location_id=LOCATION_ID,
        engine_data_type=0,
        max_documents=10,
    )

    result = str(retriever.invoke(query))
    return result

## Imrenagi.com API Client

This is the implementation of a simple HTTP client which interacts to my API server. Since the API is protected with Id Token, caller must provide the JWT bearer token on each request. To simplify this, we are using the `google-auth` library to generate the token and to manage token's lifecycle.

In [7]:
from google.oauth2 import service_account
from google.auth import compute_engine
from google.auth.transport.requests import AuthorizedSession, Request
import os

sa_path = '/home/imrenagi/projects/app/secrets/course-imrenagi.json'

class ImreNagiComAPIClient:
  def __init__(self, url="https://api-dev.imrenagi.com", service_account_key=None, aud="api"):
    self.url = url
    credentials = None   
    
    if service_account_key and os.path.exists(service_account_key):
      # Use service account credentials
      credentials = service_account.IDTokenCredentials.from_service_account_file(service_account_key, target_audience=aud)
    else:
      # Use compute engine credentials. This is used when agent is running in reasoning engine.
      request = Request()
      credentials = compute_engine.IDTokenCredentials(
          request=request, target_audience=aud, use_metadata_identity_endpoint=True
      )
      credentials.refresh(request)
    self.authed_session = AuthorizedSession(credentials)
    
  def list_courses(self):
    response = self.authed_session.get(f"{self.url}/api/v1/courses")
    return response.json()
  
  def get_course(self, course):
    response = self.authed_session.get(f"{self.url}/api/v1/courses/{course}")
    return response.json()
  
  def create_order(self, course, package, user_name, user_email):
    response = self.authed_session.post(f"{self.url}/api/v1/courses/orders", json={
      "course": course, 
      "package": package, 
      "customer": {
        "name": user_name,
        "email": user_email
      },
      "payment": {
        "method": "gopay"
      }})
    return response.json()
  
  def get_order(self, order_id):
    response = self.authed_session.get(f"{self.url}/api/v1/courses/orders/{order_id}")
    return response.json()
  
  def get_invoice(self, invoice_number):
    response = self.authed_session.get(f"{self.url}/api/v1/invoices/{invoice_number}")
    return response.json()


## Imrenagi.com API Tools

These tools below are just a langchain tools wrapper that interact with the API server.

In [8]:
import time
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def list_courses() -> List[str]:
  """List all available courses sold on the platform."""
  client = ImreNagiComAPIClient(
    service_account_key=sa_path
  )
  return client.list_courses()

In [9]:
class GetCourseInput(BaseModel):
    course: str = Field(description="name of the course. this is the unique identifier of the course. it typically contains the course title with dashes, all in lowercase.")

@tool("get-course-tool", args_schema=GetCourseInput)
def get_course(course: str) -> str:
  """Get course details by course name. course name is the unique identifier of the course. it typically contains the course title with dashes.
  This function can be used to get course details such as course packages price, etc."""
  client = ImreNagiComAPIClient(
    service_account_key=sa_path
  )
  return client.get_course(course)

In [10]:
class CreateOrderInput(BaseModel):
    course: str = Field(description="name of the course. this is the unique identifier of the course. it typically contains the course title with dashes, all in lowercase.")
    package: str = Field(description="name of the course package. this is the unique identifier of the course package. it typically contains the package title with dashes, all in lowercase.")
    user_name: str = Field(description="name of the user who is purchasing the course package.")
    user_email: str = Field(description="email of the user who is purchasing the course package.")

@tool("create-order-tool", args_schema=CreateOrderInput)
def create_order(course: str, package: str, user_name: str, user_email: str) -> str:
  """Create order for a course package. This function can be used to create an order for a course package. When this function returns successfully, it will return payment url to user to make payment. """
  client = ImreNagiComAPIClient(
    service_account_key=sa_path
  )
  
  order = client.create_order(course, package, user_name, user_email)
  print(order)
  time.sleep(5)
  
  payment_url = None
  order_number = order['number']
  for i in range(3):
    try:
      order = client.get_order(order['number'])
      invoice_number = order['payment']['invoiceNumber']
      
      invoice = client.get_invoice(invoice_number)
      payment_url = invoice['payment']['redirectUrl']
      break
    except Exception as e:
      print(f"Error getting order: {e}")
      time.sleep(5)
  
  return f"Order number {order_number} created successfully. Payment URL: {payment_url}"

In [11]:
class GetOrderInput(BaseModel):
    order_number: str = Field(description="order number identifier. this is a unique identifier in uuid format.")

@tool("get-order-tool", args_schema=GetOrderInput)
def get_order(order_number: str) -> str:
  """Get order by using order number. This function can be used to get order details such as payment status to check whether the order has been paid or not. If user already paid the course, tell them to check for their enrollment email"""
  client = ImreNagiComAPIClient(
    service_account_key=sa_path
  )
  return client.get_order(order_number)

In [12]:
# list_courses.invoke({"input":""})
# get_course.invoke({"course":"software-instrumentation"})
create_order.invoke({"course":"software-instrumentation", "package":"basic", "user_name":"Imre Nagi", "user_email":"imre.nagi2812@gmail.com"})


{'number': '09ea5dc0-461d-4454-9384-a6b69d84df75', 'course': 'software-instrumentation', 'package': 'basic', 'price': 100000, 'currency': 'IDR', 'status': 'CREATED', 'createdAt': '2024-07-13T06:33:58.162808710Z', 'paidAt': None, 'customer': {'name': 'Imre Nagi', 'email': 'imre.nagi2812@gmail.com', 'phoneNumber': '', 'shippingAddress': None, 'billingAddress': None}, 'payment': {'invoiceNumber': '', 'method': 'gopay'}, 'expiredAt': '2024-07-13T06:43:58.162809039Z', 'failedAt': None}


'Order number 09ea5dc0-461d-4454-9384-a6b69d84df75 created successfully. Payment URL: https://app.sandbox.midtrans.com/snap/v4/redirection/f2739d7b-799a-4a66-9ad9-a7d393109635'

In [15]:
# combining all tools

tools = [search_course_content, list_courses, get_course, create_order, get_order]

## Prompt

This is the prompt that will be used to interact with model. I enabled chat history and agent scratchpad so that we can pass the message history and the result from Gemini function call to the model.

In [16]:
prompt = {
    "chat_history": lambda x: x["history"],
    "input": lambda x: x["input"],
    "agent_scratchpad": (
        lambda x: format_to_openai_function_messages(x["intermediate_steps"])
    ),
} | ChatPromptTemplate(
  messages = [
    SystemMessagePromptTemplate.from_template("""
      You are a bot assistant that sells online course about software instrumentation. You only use information provided from datastore or tools. You can provide the information that is relevant to the user's question or the summary of the content. If they ask about the content, you can give them more detail about the content. If the user seems interested, you may suggest the user to enroll in the course. 
      """),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    HumanMessagePromptTemplate.from_template("Use tools to answer this questions: {input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
  ]
)

## Chat Message History

This will be used to store the chat history for each user sessions

In [17]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

## Setting up the Agent

While langchain is actually has API to create an agent, in this case we are using `reasoning_engines.LangchainAgent()` to create the agent. Once it is created, you can still use the exact same API to invoke the model as you would when you are using standard langchain agent.

In [18]:
model = LLM_MODEL

agent = reasoning_engines.LangchainAgent(
    model=model,
    tools=tools,
    prompt=prompt,    
    chat_history=get_session_history,
    agent_executor_kwargs={
      "return_intermediate_steps": True,
    },
    model_kwargs=model_kwargs,
    enable_tracing=True,
)

In [20]:
response = agent.query(
  # input="I want to purchase software instrumentation course. I want to get the basic package. My name is imre and email is imre.nagi2812@gmail.com",
  input="I have made the payment. Can you please check my order status? What's next?",
  config={"configurable": {"session_id": "demo1234"}},
  )

display(Markdown(response["output"]))

Parent run 66afc2c6-7802-4030-b99f-6e7c8d324b9b not found for run 98af0f5a-61db-4db0-a14a-6db3161d2e62. Treating as a root run.


Your order is completed. Please check your email imre.nagi2812@gmail.com for the enrollment information. 


# Deploying the Agent on Vertex AI

Deploying is as simple as calling `create()` method. We will provide the agent here and some dependencies required to run the agent.

In [21]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform==1.51.0",
        "langchain==0.1.20",
        "langchain-google-vertexai==1.0.3",
        "cloudpickle==3.0.0",
        "pydantic==2.7.1",        
        "requests==2.32.3",
        "langchain_google_community",
        "google-cloud-discoveryengine",
        "google-auth",
    ],
)

Using bucket courses-imrenagicom-agent
Writing to gs://courses-imrenagicom-agent/reasoning_engine/reasoning_engine.pkl
Writing to gs://courses-imrenagicom-agent/reasoning_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://courses-imrenagicom-agent/reasoning_engine/dependencies.tar.gz
Creating ReasoningEngine
Create ReasoningEngine backing LRO: projects/896489987664/locations/us-central1/reasoningEngines/4254318351121121280/operations/1098809898839310336
ReasoningEngine created. Resource name: projects/896489987664/locations/us-central1/reasoningEngines/4254318351121121280
To use this ReasoningEngine in another session:
reasoning_engine = vertexai.preview.reasoning_engines.ReasoningEngine('projects/896489987664/locations/us-central1/reasoningEngines/4254318351121121280')


In [19]:
# Testing the remote agent
response = remote_agent.query(
  input="Can you share about the problems discussed on final log challenge?",
  config={"configurable": {"session_id": "test1235"}},
)
display(Markdown(response["output"]))

In the final log challenge, you'll work on a demo application with some intentionally injected bugs. Your mission is to pinpoint and resolve these bugs **solely through instrumenting the app with logs, analyzing them, and interpreting the resulting metrics.**  The challenge revolves around eliminating 5xx errors and gRPC `Unknown` status codes reported during load testing. 

Let me know if you'd like a more detailed explanation of the challenge or want to explore specific aspects of it. And, if you're up for the challenge, our course can equip you with the skills to tackle this and many other real-world instrumentation scenarios! 


# Clean Up

Don't forget to clean up the resources after you are done with the agent.

In [22]:
reasoning_engines.ReasoningEngine.list()

 resource name: projects/896489987664/locations/us-central1/reasoningEngines/4254318351121121280,
 resource name: projects/896489987664/locations/us-central1/reasoningEngines/2776011773436755968]

In [25]:
remote_agent = reasoning_engines.ReasoningEngine('projects/896489987664/locations/us-central1/reasoningEngines/2776011773436755968')

remote_agent.delete()

Deleting ReasoningEngine : projects/896489987664/locations/us-central1/reasoningEngines/2776011773436755968
ReasoningEngine deleted. . Resource name: projects/896489987664/locations/us-central1/reasoningEngines/2776011773436755968
Deleting ReasoningEngine resource: projects/896489987664/locations/us-central1/reasoningEngines/2776011773436755968
Delete ReasoningEngine backing LRO: projects/896489987664/locations/us-central1/operations/2036790074355482624
ReasoningEngine resource projects/896489987664/locations/us-central1/reasoningEngines/2776011773436755968 deleted.
